In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification



c:\Users\migui\anaconda3\envs\TFG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data
data = pd.read_csv(r'C:\Users\migui\OneDrive - UFV\4ºBA\TFG\SetDatos\dota2_chat_messages.csv', encoding='utf-8')

In [3]:
print(data.head())  # Muestra las primeras filas
print(data.info())  # Información de columnas y tipos de datos
print(data.describe())  # Estadísticas de las columnas numéricas


   match        time  slot            text
0      0  1005.12122     9       ладно гг 
1      0  1005.85442     9             изи
2      0  1008.65372     9              од
3      0  1010.51992     9          ебаный
4      0  1013.91912     9  мусор на войде
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21659448 entries, 0 to 21659447
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   match   int64  
 1   time    float64
 2   slot    int64  
 3   text    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 661.0+ MB
None
              match          time          slot
count  2.165945e+07  2.165945e+07  2.165945e+07
mean   4.996186e+05  1.457327e+03  4.504226e+00
std    2.888392e+05  9.540682e+02  2.875228e+00
min    0.000000e+00 -1.068088e+03  0.000000e+00
25%    2.493490e+05  7.288557e+02  2.000000e+00
50%    4.994910e+05  1.501327e+03  5.000000e+00
75%    7.498670e+05  2.118162e+03  7.000000e+00
max    9.999990e+05  8.443546e+03  9.000000e+00


In [4]:
# Contar valores nulos por columna
valores_nulos = data.isnull().sum()
print(valores_nulos)

# Contar el total de valores nulos
total_nulos = data.isnull().sum().sum()
print("Total de valores nulos:", total_nulos)

match      0
time       0
slot       0
text     243
dtype: int64
Total de valores nulos: 243


In [5]:
# Fill NaN values with an empty string
data['text'] = data['text'].fillna('')

print('Mean length of text', data['text'].apply(lambda x: len(x)).mean())
print(data.head(15))

Mean length of text 10.344593361751416
    match        time  slot                                text
0       0  1005.12122     9                           ладно гг 
1       0  1005.85442     9                                 изи
2       0  1008.65372     9                                  од
3       0  1010.51992     9                              ебаный
4       0  1013.91912     9                      мусор на войде
5       0  1800.31402     9                              мусор 
6       0  1801.71882     9                            на войде
7       0  1802.98982     9                              репорт
8       0  1808.40822     9                                100%
9       1  -131.14018     0              twitch.tv/rage_channel
10      1  -121.60481     0  https://www.twitch.tv/rage_channel
11      1   244.47367     7                      2 даша подряд 
12      1   249.93900     7                               баша 
13      1   255.00443     4                           где даша?
1

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"using device: {device}")

devNum = torch.cuda.current_device()

print(f"device number: {devNum}")

devName = torch.cuda.get_device_name(devNum)

print(f"device name: {devName}")

using device: cuda
device number: 0
device name: NVIDIA GeForce RTX 3060 Ti


In [7]:


def translator(src: str, dest: str):
    src = src.lower()
    dest = dest.lower()
    tokenizer = AutoTokenizer\
                    .from_pretrained(f"Helsinki-NLP/opus-mt-{src}-{dest}")
    model = AutoModelForSeq2SeqLM\
                    .from_pretrained(f"Helsinki-NLP/opus-mt-{src}-{dest}")
    return pipeline(task='translation',
                    model=model, tokenizer=tokenizer, device=0) #Device 0 is the GPU




In [8]:
df_sample = data.sample(50)
translate = translator('ru', 'en')
with torch.no_grad():
    translated_text = translate([t[:100] for t in list(df_sample['text'])])
df_sample['text_en'] = [t['translation_text'] for t in translated_text]
df_sample.head(10)

c:\Users\migui\anaconda3\envs\TFG\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


,match,time,slot,text,text_en
18364267,847618,732.42120,7,oke,Okay.
8572164,394845,-5.16540,3,i did not do it. i did not. oh hi mark,i did not do it. i did not. oh hi March
16029441,739866,1338.43988,7,peruano y la conchadetuputamadre,peruano y la conchadetuputamadre
19519371,900916,2033.18002,2,Ez,Ez
5873479,270621,-17.79567,5,?,"♪ ♪ ♪ Oh, oh, oh, oh, oh, oh, oh, oh, oh, oh, ..."
2457656,113034,2103.91397,2,у нас файт он крипов пиздит,We've got a fat he's fucking screaming.
3021867,138828,1839.08910,1,all normal,all nonmal
10006457,461101,1306.51631,4,wow,Wow.
2194111,100889,2460.32478,2,изи,Easy
9619798,443212,408.63360,9,no fun,no fun


In [9]:
data_translate = data.head(25000)
translate = translator('ru', 'en')
with torch.no_grad():
    translated_text = translate([t[:100] for t in list(data_translate['text'])])
data_translate['text_en'] = [t['translation_text'] for t in translated_text]
data_translate.head(10)

c:\Users\migui\anaconda3\envs\TFG\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Users\migui\AppData\Local\Temp\ipykernel_25184\3408897131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_translate['text_en'] = [t['translation_text'] for t in translated_text]


,match,time,slot,text,text_en
0,0,1005.12122,9,ладно гг,Okay.
1,0,1005.85442,9,изи,Easy
2,0,1008.65372,9,од,"- Yeah, yeah, yeah, yeah, yeah, yeah, yeah, ye..."
3,0,1010.51992,9,ебаный,You're fucking kidding me.
4,0,1013.91912,9,мусор на войде,There's garbage in the door.
5,0,1800.31402,9,мусор,rubbish
6,0,1801.71882,9,на войде,at the entrance
7,0,1802.98982,9,репорт,Report
8,0,1808.40822,9,100%,100%
9,1,-131.14018,0,twitch.tv/rage_channel,Twitch.tv/range_channel


In [10]:
# Guardar el DataFrame en un archivo CSV
data_translate.to_csv('data_translate.csv', index=False)
